In [87]:
import numpy as np
import baseline_model
from casadi import *
import do_mpc
import decentralized as dec
import util
import sim_drone_main
from baseline_model import baseline_drone_model
from baseline_model_mpc import baseline_drone_mpc
from baseline_model_simulator import baseline_drone_simulator
import itertools

In [88]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
Q = np.eye(6)
R = np.eye(3)
n_agents = 3
n_inputs = 3
n_states = 6
Qf = Q*1e3

In [4]:
Qs = [Q] * n_agents
Rs = [R] * n_agents
Qfs = [Qf] * n_agents

In [143]:
model_type = 'continuous' # either 'discrete' or 'continuous'
model = do_mpc.model.Model(model_type)

x = model.set_variable(var_type='_x', var_name='x', shape=(6, 1))
u = model.set_variable(var_type='_u', var_name='u', shape=(3, 1))
print(f'Shape of x is {x.shape}')
print(f'Shape of u is {u.shape}')
#x = p_x,p_y,p_z,v_x,v_y,v_z
#u = theta, phi, tau
g = 9.81
model.set_rhs('x', vertcat(x[3], x[4], x[5], g*np.tan(u[0]), -g*np.tan(u[1]), u[2]-g))

Shape of x is (6, 1)
Shape of u is (3, 1)


In [144]:
theta_max = np.pi/6
phi_max = np.pi/6
tau_max = 5
v_max = 5

x_dims = [6,6,6]

Q = np.eye(n_states)*10
Qf = np.eye(n_states)*1e3
R = np.eye(n_inputs)

radius = 0.5
n_dims = [3,3,3]
episode=200
x_baseline_init, x_baseline_f = util.paper_setup_3_quads() 
#concatenated initial and final states of all 3 agents

x_baseline1 = x_baseline_init

states_list = np.zeros((episode+1,9)) #positions of each drone
states_list[0,:] = np.array([x_baseline1[0],x_baseline1[1],x_baseline1[2],\
                        x_baseline1[6],x_baseline1[7],x_baseline1[8],\
                        x_baseline1[12],x_baseline1[13],x_baseline1[14]]).flatten()

In [153]:
baseline_drone_model(dec.split_agents(x_baseline_f.reshape(1,-1),x_dims)[0].flatten(), Q, R, Qf)

Shape of x is (6, 1)
Shape of u is (3, 1)


In [178]:
model_baseline = [baseline_drone_model(dec.split_agents(x_baseline_f.reshape(1,-1),x_dims)[i].flatten(), Q, R, Qf, x_baseline1, x_dims) for i in range(n_agents)]

Shape of x is (6, 1)
Shape of u is (3, 1)
Shape of x is (6, 1)
Shape of u is (3, 1)
Shape of x is (6, 1)
Shape of u is (3, 1)


In [180]:
mpc_baseline = [baseline_drone_mpc(model_i,n_agents,x_baseline1,x_dims, v_max, theta_max, phi_max, tau_max) for model_i in model_baseline]

/home/randy233/anaconda3/envs/Randy/lib/python3.10/site-packages/do_mpc/controller.py:904: UserWarning: rterm was not set and defaults to zero. Changes in the control inputs are not penalized. Can lead to oscillatory behavior.
  warnings.warn('rterm was not set and defaults to zero. Changes in the control inputs are not penalized. Can lead to oscillatory behavior.')


In [181]:
len(mpc_baseline)

3

In [182]:
simulator_baseline = [baseline_drone_simulator(model_i) for model_i in model_baseline]

In [190]:
dec.split_agents(x_baseline1.reshape(1,-1),x_dims)[0]

array([[0.491, 1.501, 0.984, 0.   , 0.   , 0.   ]])

In [192]:
simulator_baseline

In [198]:
split_states = dec.split_agents(x_baseline1.reshape(1,-1),x_dims)
for m in range(len(simulator_baseline)):
        simulator_baseline[m].x0['x'] = split_states[m].T #dimension mismatch here?
        mpc_baseline[m].x0 = split_states[m].T

In [205]:
u_init_baseline = np.full((n_agents*n_inputs,1), 0.0)
u_dims = [3,3,3]

In [207]:
split_inputs = dec.split_agents(u_init_baseline.reshape(1,-1),u_dims)
split_inputs

[array([[0., 0., 0.]]), array([[0., 0., 0.]]), array([[0., 0., 0.]])]

In [212]:
for m in range(len(mpc_baseline)):
    print(split_inputs[m].T.shape)

(3, 1)
(3, 1)
(3, 1)


In [224]:
u0_baseline = []
x_baseline_next = []
for m in range(len(mpc_baseline)):
    mpc_baseline[m].u0 = split_inputs[m].T
    simulator_baseline[m].u0 = split_inputs[m].T
    mpc_baseline[m].set_initial_guess()
    u0_baseline.append(mpc_baseline[m].make_step(split_states[m].T))
    x_baseline_next.append(simulator_baseline[m].make_step(split_inputs[m].T))

This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:      666
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       51

Total number of variables............................:      267
                     variables with only lower bounds:        0
                variables with lower and upper bounds:       15
                     variables with only upper bounds:        0
Total number of equality constraints.................:      216
Total number of inequality constraints...............:       15
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:       15

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  9.1634593e+03 6.50e-03 8.42e+00  -1.0 0.00e+00    -  0.00e+00 0.00e+00  

In [228]:
[mpc_baseline[i].data._lam_g_num for i in range(len(mpc_baseline))][0]

array([[ 3.322e+03, -1.098e+00,  1.225e+03,  7.795e+02, -6.229e-02,
         5.516e+03, -2.462e+03,  7.883e-01, -9.190e+02, -5.436e+02,
         3.358e-02, -4.122e+03, -8.206e+02,  2.628e-01, -3.063e+02,
        -1.538e+02,  2.435e-03, -1.364e+03,  3.282e+03, -1.051e+00,
         1.225e+03,  6.153e+02, -9.741e-03,  5.455e+03, -2.462e+03,
         7.883e-01, -9.190e+02, -4.205e+02, -5.832e-03, -4.076e+03,
        -8.206e+02,  2.628e-01, -3.063e+02, -1.128e+02, -1.070e-02,
        -1.348e+03,  3.282e+03, -1.051e+00,  1.225e+03,  4.512e+02,
         4.281e-02,  5.394e+03, -3.282e+03,  1.051e+00, -1.225e+03,
        -4.512e+02, -4.281e-02, -5.394e+03,  1.003e+01,  1.003e+01,
         1.003e+01, -2.432e+03,  7.566e-01, -9.186e+02, -3.064e+02,
         2.559e-02, -4.023e+03, -8.107e+02,  2.522e-01, -3.062e+02,
        -7.510e+01,  1.231e-04, -1.331e+03,  3.243e+03, -1.009e+00,
         1.225e+03,  3.004e+02, -4.923e-04,  5.323e+03, -2.432e+03,
         7.566e-01, -9.186e+02, -1.848e+02, -1.2

In [229]:
theta_max = np.pi/6
phi_max = np.pi/6
tau_max = 5
v_max = 5

x_dims = [n_states]*n_agents
u_dims = [n_inputs]*n_agents

Q = np.eye(n_states)*10
Qf = np.eye(n_states)*1e3
R = np.eye(n_inputs)

radius = 0.5
n_dims = [3,3,3]
episode= 200
x_baseline_init, x_baseline_f = util.paper_setup_3_quads()

x_baseline1 = x_baseline_init #concatenated states of all agents

states_list = np.zeros((episode+1,3*n_agents)) #positions of each drone
states_list[0,:] = np.array([x_baseline1[0],x_baseline1[1],x_baseline1[2],\
                        x_baseline1[6],x_baseline1[7],x_baseline1[8],\
                        x_baseline1[12],x_baseline1[13],x_baseline1[14]]).flatten()

In [233]:
n_agents = 3
n_states = 6
n_inputs = 3

def setup_baseline(x_baseline, x_baseline_f, v_max, theta_max, phi_max, tau_max,\
                    x_dims, u_dims, Q, R, Qf, n_agents, n_dims, radius):
    model_baseline = [baseline_drone_model(dec.split_agents(x_baseline_f.reshape(1,-1),x_dims)[i].flatten(), Q, \
                     R, Qf, x_baseline, x_dims) for i in range(n_agents)] 
    #a list of baseline models for each agent
    
    mpc_baseline = [baseline_drone_mpc(model_i,n_agents,x_baseline, x_dims, v_max, theta_max, phi_max,\
                   tau_max) for model_i in model_baseline]
    #a list of baseline mpc controllers for each agent
    
    simulator_baseline = [baseline_drone_simulator(model_i) for model_i in model_baseline]
    
    #splitting the states of each agent
    split_states = dec.split_agents(x_baseline.reshape(1,-1),x_dims)

    for m in range(len(simulator_baseline)):
        
        simulator_baseline[m].x0['x'] = split_states[m].T #dimension mismatch here?
        mpc_baseline[m].x0 = split_states[m].T

    u_init_baseline = np.full((n_agents*n_inputs,1), 0.0)
    split_inputs = dec.split_agents(u_init_baseline.reshape(1,-1),u_dims)
    
    u0_baseline = []
    x_baseline_next = []
    for m in range(len(mpc_baseline)):
        mpc_baseline[m].u0 = split_inputs[m].T
        simulator_baseline[m].u0 = split_inputs[m].T
        mpc_baseline[m].set_initial_guess()
        u0_baseline.append(mpc_baseline[m].make_step(split_states[m].T))
        x_baseline_next.append(simulator_baseline[m].make_step(split_inputs[m].T))

    return [u0_baseline, x_baseline_next, [mpc_baseline[i].data._lam_g_num for i in range(len(mpc_baseline))]] 
    #last _lam_g_num is the Lagrange multiplier

In [239]:
import concurrent.futures
import time

In [240]:
time_start = time.perf_counter()
with concurrent.futures.ProcessPoolExecutor() as executor:
        for k in range(50):
            f = executor.submit(setup_baseline(x_baseline1, x_baseline_f, v_max, theta_max, phi_max, tau_max,\
        x_dims, u_dims, Q, R, Qf, n_agents, n_dims, radius), x_baseline1, x_baseline_f, 100)
time_finish = time.perf_counter()
print("Total time: ", time_finish - time_start)

Shape of x is (6, 1)
Shape of u is (3, 1)
Shape of x is (6, 1)
Shape of u is (3, 1)
Shape of x is (6, 1)
Shape of u is (3, 1)
This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:      666
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       51

Total number of variables............................:      267
                     variables with only lower bounds:        0
                variables with lower and upper bounds:       15
                     variables with only upper bounds:        0
Total number of equality constraints.................:      216
Total number of inequality constraints...............:       15
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:       15

iter    objective    inf_pr   inf

In [250]:
f.result()

TypeError: 'list' object is not callable